In [2]:
import requests
import pandas as pd
country = "italy"

In [12]:
import requests
url = 'https://en.wikipedia.org/wiki/Template:COVID-19_pandemic_data/Italy_medical_cases'
website_url = requests.get('https://en.wikipedia.org/wiki/Template:COVID-19_pandemic_data/Italy_medical_cases').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,"html.parser")

My_table = soup.find_all('table',{'class':'wikitable sortable'})

In [14]:
My_table.__len__()

1

In [5]:
df=pd.read_html(str(My_table))

In [7]:
df=pd.DataFrame(df[0])

In [17]:
df = pd.read_html(url, attrs={"class": "wikitable mw-collapsible"})

In [22]:
df = df[0]

In [25]:
df.head(10)

Date North-West                    North-East                         \
         Date        VDA  LIG  PIE      LOM        VEN   TN   BZ  FVG     EMR   
0  2020-01-31        NaN  NaN  NaN      NaN        NaN  NaN  NaN  NaN     NaN   
1  2020-02-06        NaN  NaN  NaN      NaN        NaN  NaN  NaN  NaN     NaN   
2  2020-02-21        NaN  NaN  NaN       15      (1) 2  NaN  NaN  NaN     NaN   
3  2020-02-22        NaN  NaN    1   (1) 40         16  NaN  NaN  NaN       2   
4  2020-02-23        NaN  NaN    2   (1) 57          7  NaN  NaN  NaN       7   
5  2020-02-24        NaN  NaN  NaN   (4) 61          7  NaN  NaN  NaN       9   
6  2020-02-25        NaN    1  NaN   (2) 67     (1) 11  NaN    1  NaN       8   
7  2020-02-26        NaN   10  NaN   (1) 65         28  NaN  NaN  NaN  (1) 21   
8  2020-02-27        NaN    8    1   (5) 98         40  NaN  NaN  NaN      50   
9  2020-02-28        NaN  NaN    9  (3) 128         40  NaN  NaN  NaN  (1) 48   

   ... Confirmed Deaths       Active       Recoveries         Tests      Refs  \
   ...     Total    New Total    ICU Total        New Total   Total      Refs   
0  ...         2      –     –      –     2          –     –     NaN       [2]   
1  ...         3      –     –      –     3          –     –     NaN       [3]   
2  ...        20      1     1      –    19          –     –     NaN       [4]   
3  ...        79      1     2     18    75          –     –     NaN    [5][6]   
4  ...       150      1     3     25   147          –     –     NaN    [7][8]   
5  ...       227      4     7     27   199          1     1     NaN   [9][10]   
6  ...       320      3    10     35   309          –     1  >8,600  [11][12]   
7  ...       445      2    12     36   430          2     3    9587  [13][14]   
8  ...       650      5    17     56   588         42    45   12014  [16][17]   
9  ...       888      4    21     64   821          1    46   15695  [19][20]   

       Ns  
       Ns  
0     NaN  
1     [a]  
2     NaN  
3     NaN  
4     [b]  
5     [b]  
6     [b]  
7  [b][c]  
8     [d]  
9     NaN  

[10 rows x 33 columns]

In [ ]:
# aggressive data cleaning